# ReduceLROnPlateau学习率动态调整

## 核心概念

### 学习率调整的重要性

学习率是深度学习中最关键的超参数之一：
- **过大**: 导致训练不稳定，损失震荡甚至发散
- **过小**: 训练速度慢，容易陷入局部最优
- **动态调整**: 训练初期用大学习率快速下降，后期用小学习率精细调优

### ReduceLROnPlateau原理

当监控指标在一定epoch内不再改善时，自动降低学习率：
1. 监控验证集指标(如val_loss)
2. 连续N个epoch无改善时触发
3. 按照指定比例降低学习率
4. 重复以上过程直到学习率达到下限

## 应用场景

1. **训练后期优化**: 当损失降低缓慢时降低学习率进行精细调整
2. **避免过拟合**: 小学习率有助于模型更平滑地收敛
3. **自适应训练**: 无需手动设置学习率衰减策略
4. **迁移学习**: 微调预训练模型时动态调整学习率

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os

## 1. 准备数据集

使用Fashion MNIST数据集进行演示。

In [ ]:
# 加载Fashion MNIST数据集
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

# 数据预处理
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# 添加通道维度
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# 转换标签为one-hot编码
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

print(f'训练集形状: {x_train.shape}, 标签形状: {y_train.shape}')
print(f'测试集形状: {x_test.shape}, 标签形状: {y_test.shape}')

# Fashion MNIST类别
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

## 2. 构建模型

构建一个卷积神经网络用于图像分类。

In [ ]:
def create_model(learning_rate=0.001):
    """
    创建CNN模型
    
    参数:
        learning_rate: 初始学习率
    
    返回:
        编译好的模型
    """
    model = keras.Sequential([
        # 第一个卷积块
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        
        # 第二个卷积块
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        
        # 第三个卷积块
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        
        # 全连接层
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])
    
    # 使用指定学习率的优化器
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

model = create_model()
model.summary()

## 3. ReduceLROnPlateau详解

### 关键参数说明

- `monitor`: 监控的指标名称(通常是'val_loss')
- `factor`: 学习率衰减因子，新学习率 = 旧学习率 × factor
- `patience`: 容忍多少个epoch无改善
- `min_delta`: 最小改善幅度
- `cooldown`: 降低学习率后等待多少个epoch再继续监控
- `min_lr`: 学习率下限
- `mode`: 'min'(指标越小越好)或'max'(指标越大越好)
- `verbose`: 日志详细程度

In [ ]:
# 策略1: 标准配置
reduce_lr_standard = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,              # 每次降低50%
    patience=3,              # 3个epoch无改善后降低
    min_delta=0.001,
    min_lr=1e-7,
    verbose=1
)

# 策略2: 激进策略(快速降低学习率)
reduce_lr_aggressive = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,              # 每次降低80%
    patience=2,              # 2个epoch后就降低
    min_delta=0.0001,
    min_lr=1e-7,
    verbose=1
)

# 策略3: 保守策略(缓慢降低学习率)
reduce_lr_conservative = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.7,              # 每次降低30%
    patience=5,              # 5个epoch后才降低
    min_delta=0.01,
    min_lr=1e-7,
    verbose=1
)

# 策略4: 带cooldown的策略
reduce_lr_cooldown = keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    cooldown=2,              # 降低学习率后等待2个epoch
    min_delta=0.001,
    min_lr=1e-7,
    verbose=1
)

print('ReduceLROnPlateau回调函数配置完成')

## 4. 自定义学习率监控回调

创建一个回调函数来记录学习率变化。

In [ ]:
class LearningRateMonitor(keras.callbacks.Callback):
    """
    监控和记录学习率变化
    
    功能:
    1. 记录每个epoch的学习率
    2. 检测学习率变化
    3. 显示学习率调整信息
    """
    
    def __init__(self):
        super().__init__()
        self.learning_rates = []
        self.epochs = []
    
    def on_epoch_begin(self, epoch, logs=None):
        """记录当前学习率"""
        lr = float(keras.backend.get_value(self.model.optimizer.learning_rate))
        self.learning_rates.append(lr)
        self.epochs.append(epoch)
        
        # 检测学习率变化
        if len(self.learning_rates) > 1:
            prev_lr = self.learning_rates[-2]
            if lr != prev_lr:
                change_ratio = (lr - prev_lr) / prev_lr * 100
                print(f'\n学习率已调整: {prev_lr:.2e} -> {lr:.2e} ({change_ratio:+.1f}%)')
    
    def on_train_end(self, logs=None):
        """训练结束时总结学习率变化"""
        print(f'\n学习率变化总结:')
        print(f'  初始学习率: {self.learning_rates[0]:.2e}')
        print(f'  最终学习率: {self.learning_rates[-1]:.2e}')
        print(f'  调整次数: {len(set(self.learning_rates)) - 1}')

## 5. 实验1: 无学习率调整的基准

先训练一个固定学习率的模型作为对照。

In [ ]:
print('\n========== 实验1: 固定学习率基准 ==========')
print('学习率: 0.001 (固定)\n')

model_baseline = create_model(learning_rate=0.001)
lr_monitor_baseline = LearningRateMonitor()

history_baseline = model_baseline.fit(
    x_train, y_train,
    batch_size=128,
    epochs=3,  # 测试用，实际训练可设为20
    validation_split=0.2,
    callbacks=[lr_monitor_baseline],
    verbose=1
)

# 评估模型
test_loss, test_acc = model_baseline.evaluate(x_test, y_test, verbose=0)
print(f'\n测试结果 - 损失: {test_loss:.4f}, 准确率: {test_acc:.4f}')

## 6. 实验2: 标准ReduceLROnPlateau

使用标准配置的学习率调整策略。

In [ ]:
print('\n========== 实验2: 标准ReduceLROnPlateau ==========')
print('初始学习率: 0.001')
print('策略: factor=0.5, patience=3\n')

model_standard = create_model(learning_rate=0.001)
lr_monitor_standard = LearningRateMonitor()

callbacks_standard = [
    reduce_lr_standard,
    lr_monitor_standard
]

history_standard = model_standard.fit(
    x_train, y_train,
    batch_size=128,
    epochs=3,  # 测试用
    validation_split=0.2,
    callbacks=callbacks_standard,
    verbose=1
)

# 评估模型
test_loss, test_acc = model_standard.evaluate(x_test, y_test, verbose=0)
print(f'\n测试结果 - 损失: {test_loss:.4f}, 准确率: {test_acc:.4f}')

## 7. 实验3: 激进策略

使用激进的学习率衰减策略。

In [ ]:
print('\n========== 实验3: 激进策略 ==========')
print('初始学习率: 0.001')
print('策略: factor=0.2, patience=2\n')

model_aggressive = create_model(learning_rate=0.001)
lr_monitor_aggressive = LearningRateMonitor()

callbacks_aggressive = [
    reduce_lr_aggressive,
    lr_monitor_aggressive
]

history_aggressive = model_aggressive.fit(
    x_train, y_train,
    batch_size=128,
    epochs=3,  # 测试用
    validation_split=0.2,
    callbacks=callbacks_aggressive,
    verbose=1
)

# 评估模型
test_loss, test_acc = model_aggressive.evaluate(x_test, y_test, verbose=0)
print(f'\n测试结果 - 损失: {test_loss:.4f}, 准确率: {test_acc:.4f}')

## 8. 实验4: 保守策略

使用保守的学习率衰减策略。

In [ ]:
print('\n========== 实验4: 保守策略 ==========')
print('初始学习率: 0.001')
print('策略: factor=0.7, patience=5\n')

model_conservative = create_model(learning_rate=0.001)
lr_monitor_conservative = LearningRateMonitor()

callbacks_conservative = [
    reduce_lr_conservative,
    lr_monitor_conservative
]

history_conservative = model_conservative.fit(
    x_train, y_train,
    batch_size=128,
    epochs=3,  # 测试用
    validation_split=0.2,
    callbacks=callbacks_conservative,
    verbose=1
)

# 评估模型
test_loss, test_acc = model_conservative.evaluate(x_test, y_test, verbose=0)
print(f'\n测试结果 - 损失: {test_loss:.4f}, 准确率: {test_acc:.4f}')

## 9. 组合使用：ReduceLROnPlateau + EarlyStopping

在实际应用中，通常将学习率调整与早停机制结合使用。

In [ ]:
print('\n========== 实验5: 组合策略 ==========')
print('ReduceLROnPlateau + EarlyStopping + ModelCheckpoint\n')

model_combined = create_model(learning_rate=0.001)
lr_monitor_combined = LearningRateMonitor()

# 创建保存目录
checkpoint_dir = 'lr_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

callbacks_combined = [
    # 学习率调整
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    
    # 早停机制
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=7,              # 比ReduceLR的patience大
        restore_best_weights=True,
        verbose=1
    ),
    
    # 模型保存
    keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(checkpoint_dir, 'best_model.keras'),
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    ),
    
    # 学习率监控
    lr_monitor_combined
]

history_combined = model_combined.fit(
    x_train, y_train,
    batch_size=128,
    epochs=3,  # 测试用
    validation_split=0.2,
    callbacks=callbacks_combined,
    verbose=1
)

# 评估模型
test_loss, test_acc = model_combined.evaluate(x_test, y_test, verbose=0)
print(f'\n测试结果 - 损失: {test_loss:.4f}, 准确率: {test_acc:.4f}')

## 10. 对比分析

对比不同学习率策略的效果。

In [ ]:
def compare_experiments(histories, names):
    """
    对比多个实验的结果
    """
    print('\n========== 实验对比 ==========')
    print(f'{"策略":<15} {"最佳验证损失":<15} {"最佳验证准确率":<15} {"最终验证准确率"}')
    print('-' * 60)
    
    results = []
    for history, name in zip(histories, names):
        best_val_loss = min(history.history['val_loss'])
        best_val_acc = max(history.history['val_accuracy'])
        final_val_acc = history.history['val_accuracy'][-1]
        
        print(f'{name:<15} {best_val_loss:<15.4f} {best_val_acc:<15.4f} {final_val_acc:.4f}')
        
        results.append({
            'name': name,
            'best_val_loss': best_val_loss,
            'best_val_acc': best_val_acc,
            'final_val_acc': final_val_acc
        })
    
    # 找出最佳策略
    best_strategy = max(results, key=lambda x: x['best_val_acc'])
    print(f'\n最佳策略: {best_strategy["name"]} (验证准确率: {best_strategy["best_val_acc"]:.4f})')
    
    return results

# 对比所有实验
all_histories = [
    history_baseline,
    history_standard,
    history_aggressive,
    history_conservative,
    history_combined
]

all_names = [
    '固定学习率',
    '标准策略',
    '激进策略',
    '保守策略',
    '组合策略'
]

results = compare_experiments(all_histories, all_names)

## 11. 学习率变化可视化

可视化不同策略下的学习率变化轨迹。

In [ ]:
def visualize_learning_rates(lr_monitors, names):
    """
    可视化学习率变化
    """
    print('\n========== 学习率变化轨迹 ==========')
    
    for monitor, name in zip(lr_monitors, names):
        print(f'\n{name}:')
        print(f'  Epochs: {monitor.epochs}')
        print(f'  学习率: {[f"{lr:.2e}" for lr in monitor.learning_rates]}')
        
        # 统计信息
        unique_lrs = len(set(monitor.learning_rates))
        print(f'  不同学习率数量: {unique_lrs}')
        if unique_lrs > 1:
            reduction_ratio = monitor.learning_rates[-1] / monitor.learning_rates[0]
            print(f'  总衰减比例: {reduction_ratio:.4f}')

# 可视化所有实验的学习率变化
all_monitors = [
    lr_monitor_baseline,
    lr_monitor_standard,
    lr_monitor_aggressive,
    lr_monitor_conservative,
    lr_monitor_combined
]

visualize_learning_rates(all_monitors, all_names)

## 12. 清理临时文件

In [ ]:
import shutil

# 清理测试产生的文件和目录
if os.path.exists(checkpoint_dir):
    shutil.rmtree(checkpoint_dir)
    print(f'已删除目录: {checkpoint_dir}')

print('清理完成')

## 总结

### ReduceLROnPlateau最佳实践

1. **参数选择建议**:
   - `factor`: 通常设为0.5或0.1
     - 0.5: 温和降低，适合大多数场景
     - 0.1: 激进降低，适合快速收敛
   - `patience`: 根据数据集大小调整
     - 小数据集: 2-3
     - 大数据集: 5-10
   - `min_lr`: 建议设为初始学习率的1/1000到1/10000

2. **监控指标选择**:
   - 优先监控`val_loss`而非`val_accuracy`
   - 损失曲线更平滑，更容易判断plateau
   - 准确率在高值时变化不明显

3. **与其他回调组合**:
   ```python
   callbacks = [
       ReduceLROnPlateau(patience=3),    # 学习率调整
       EarlyStopping(patience=10),       # patience应大于ReduceLR
       ModelCheckpoint(...)              # 保存最佳模型
   ]
   ```

4. **常见问题**:
   - **学习率降低过快**: 增大patience或factor
   - **训练停滞**: 检查min_lr是否过大
   - **过早停止**: EarlyStopping的patience应大于ReduceLR

### 与其他学习率策略对比

| 策略 | 优点 | 缺点 | 适用场景 |
|------|------|------|----------|
| ReduceLROnPlateau | 自适应，无需预设 | 可能反应滞后 | 训练时间充足 |
| CosineAnnealing | 周期性恢复，防止局部最优 | 需要预知总epoch | 研究实验 |
| ExponentialDecay | 平滑衰减 | 需要调整衰减率 | 稳定收敛 |
| StepDecay | 简单可控 | 需要手动设置步数 | 经验丰富时 |

### 高级技巧

1. **Warmup + ReduceLR**: 训练初期线性增加学习率，后期动态降低
2. **Cyclical LR**: 周期性地调整学习率，有助于跳出局部最优
3. **Per-parameter LR**: 对不同层使用不同的学习率策略
4. **Gradient Clipping**: 配合学习率调整，防止梯度爆炸

### 调试建议

1. 使用TensorBoard可视化学习率和损失曲线
2. 记录每次学习率变化的epoch和对应的指标值
3. 对比不同配置下的训练曲线
4. 保存学习率变化历史用于后续分析